In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR='/home/csunix/ml17z2x/Downloads/practice/cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['readme.html', 'data_batch_2', 'batches.meta', 'data_batch_1', 'data_batch_3', 'data_batch_4', 'test_batch', 'data_batch_5']


In [2]:
def load_data(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'], data[b'labels']


In [3]:
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        #print(all_data)            
        self._data = np.vstack(all_data)
        #print(self._data)
        
        self._data = self._data / 255
        #print(self._data)
        self._labels = np.hstack(all_labels)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        
        #print(self._data.shape)
        #print(self._labels.shape)
        if self._need_shuffle:
            self._shuffle_data()
    
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        #print(self._indicator)
        #print(batch_size)
        if end_indicator > self._num_examples:
            #print(end_indicator)
            #print(self._num_examples)
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
                
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
            
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range (1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

#print(type(train_filenames))
#print(train_filenames)
train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

#batch_data, batch_labels = train_data.next_batch(100)
#print(batch_data)
#print(batch_labels)

In [5]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 3072]) #placeholder 占位图  ,None 表示样本数不确定
y = tf.placeholder(tf.int64,[None])

hidden1 = tf.layers.dense(x, 100, activation=tf.nn.relu)
hidden2 = tf.layers.dense(hidden1, 100, activation=tf.nn.relu)
hidden3 = tf.layers.dense(hidden2, 50, activation=tf.nn.relu)
y_ = tf.layers.dense(hidden3,10)

loss = tf.losses.sparse_softmax_cross_entropy(labels= y, logits= y_)

predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))



with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)



In [6]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 1000000
test_steps = 100


with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict={x: batch_data, y: batch_labels})
        
        if (i+1) % 500 == 0:
            print('[Train]Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
            
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test]Step: %d, acc: %4.5f' % (i+1,test_acc))

[Train]Step: 500, loss: 2.21339, acc: 0.30000
[Train]Step: 1000, loss: 1.71605, acc: 0.30000
[Train]Step: 1500, loss: 1.94759, acc: 0.15000
[Train]Step: 2000, loss: 1.86528, acc: 0.30000
[Train]Step: 2500, loss: 1.67010, acc: 0.35000
[Train]Step: 3000, loss: 1.59634, acc: 0.40000
[Train]Step: 3500, loss: 1.40709, acc: 0.60000
[Train]Step: 4000, loss: 1.75244, acc: 0.40000
[Train]Step: 4500, loss: 1.89081, acc: 0.30000
[Train]Step: 5000, loss: 1.99356, acc: 0.40000
[Test]Step: 5000, acc: 0.38150
[Train]Step: 5500, loss: 1.42318, acc: 0.45000
[Train]Step: 6000, loss: 1.95462, acc: 0.30000
[Train]Step: 6500, loss: 1.57537, acc: 0.40000
[Train]Step: 7000, loss: 1.37747, acc: 0.50000
[Train]Step: 7500, loss: 2.03621, acc: 0.25000
[Train]Step: 8000, loss: 1.62171, acc: 0.35000
[Train]Step: 8500, loss: 1.43404, acc: 0.40000
[Train]Step: 9000, loss: 1.47939, acc: 0.40000
[Train]Step: 9500, loss: 1.41169, acc: 0.55000
[Train]Step: 10000, loss: 1.55582, acc: 0.50000
[Test]Step: 10000, acc: 0.435

[Train]Step: 81000, loss: 1.29593, acc: 0.55000
[Train]Step: 81500, loss: 1.79646, acc: 0.30000
[Train]Step: 82000, loss: 1.21147, acc: 0.55000
[Train]Step: 82500, loss: 1.41863, acc: 0.65000
[Train]Step: 83000, loss: 1.05510, acc: 0.75000
[Train]Step: 83500, loss: 1.05203, acc: 0.55000
[Train]Step: 84000, loss: 1.15357, acc: 0.65000
[Train]Step: 84500, loss: 1.84265, acc: 0.45000
[Train]Step: 85000, loss: 1.90633, acc: 0.35000
[Test]Step: 85000, acc: 0.46600
[Train]Step: 85500, loss: 1.55087, acc: 0.40000


KeyboardInterrupt: 